In [230]:
from manim import *
import numpy as np
from typing import List, TypeVar

quality = "ql"

In [231]:
# Font sizes
SMALL_FONT = 24
MED_FONT = 32
LARGE_FONT = 48

# Colors
C_HIGHLIGHT = TEAL_C
C_EMPHASIS = ORANGE

In [232]:
from math import acos, atan2, cos, sin

def squared_norm(vector: np.array) -> float:
    return np.dot(vector, vector)

def norm(vector: np.array) -> float:
    """
    Returns the length of a vector.
    """
    return squared_norm(vector) ** 0.5


def normalize(vector: np.array) -> np.array:
    """
    Returns the vector normalized so that it has length 1.
    """
    return vector / norm(vector)

def circle_to_circle_tangent(center1: np.array, radius1: float, center2: np.array, radius2: float) -> List[np.array]:
    """
    Returns the outer tangent line between two circles.
    The tangent is such that it is on the outside of 1 and 2 when they are arranged in counter-clockwise fashion.
    """
    radius1 *= -1
    radius2 *= -1
    dist = norm(center2 - center1)

    delta = (center2 - center1) / dist
    cross = np.array([-delta[1], delta[0], 0])
    alpha = (radius1 - radius2) / dist
    beta = (1 - alpha ** 2) ** 0.5
    return [
        center1 + (alpha * delta + beta * cross) * radius1,
        center2 + (alpha * delta + beta * cross) * radius2
    ]

def circle_to_point_tangent(center: np.array, radius: float, point: np.array) -> np.array:
    dist = norm(point - center)
    angle = acos(radius / dist)
    angle_offset = atan2(point[1] - center[1], point[0] - center[0])
    return np.array([
        center[0] + radius * cos(angle_offset - angle),
        center[1] + radius * sin(angle_offset - angle),
        0
    ])

def circle_to_point_tangent(point: np.array, center: np.array, radius: float) -> List[float]:
    return circle_to_circle_tangent(center, -radius, point)

In [233]:

T = TypeVar('T')
class PlateCircle(VGroup):
    def __init__(self, radius: float, offset: float) -> None:
        self._radius = radius
        self._offset = offset

        self._inner_circle = Circle(self.inner_radius())
        self._outer_circle = Circle(self.outer_radius())
        super().__init__(self._inner_circle, self._outer_circle)

    @staticmethod
    def tangent_points(start, end) -> List[np.array]:
        return circle_to_circle_tangent(start.center(), start.outer_radius(), end.center(), end.outer_radius())

    @staticmethod
    def tangent_line(start, end) -> List[np.array]:
        return Line(*PlateCircle.tangent_points(start, end))

    def inner_circle(self):
        return self._inner_circle

    def outer_circle(self):
        return self._outer_circle

    def center(self) -> np.array:
        return self.inner_circle().get_center()
    
    def inner_radius(self) -> float:
        return self._radius
    
    def outer_radius(self) -> float:
        return self._radius + self._offset
    
    def copy(self: T, center: np.array) -> T:
        return super().copy().move_to(center)
    
    def draw_inner_circle(self) -> Animation:
        return GrowFromCenter(self.inner_circle())
    
    def draw_outer_circle(self) -> Animation:
        return GrowFromCenter(self.outer_circle())

In [292]:
class Plate(VGroup):
    def __init__(self, points: List[PlateCircle], boundary_order: List[int]) -> None:
        self._points = points
        self._boundary = [points[i] for i in boundary_order]
        # self._boundary = list(filter(lambda p: p not in self._inside, self._points))
        self._boundary_segments = self._make_boundary_segments()

        super().__init__(*[*self._points, *self._boundary_segments])

    def _make_boundary_segments(self) -> List[Line]:
        return [PlateCircle.tangent_line(self._boundary[i - 1], self._boundary[i]) for i in range(len(self._boundary))]
    
    def draw_inner_circles(self) -> Animation:
        return AnimationGroup(*[x.draw_inner_circle() for x in self._points], lag_ratio=0.75)

    def draw_outer_circles(self) -> Animation:
        return AnimationGroup(*[x.draw_outer_circle() for x in self._points], lag_ratio=0.75)
    
    def draw_boundary(self) -> Animation:
        return AnimationGroup(*[Create(x, run_time=1.25) for x in self._boundary_segments], lag_ratio=0.75)

The reccomend way to create plates is by first drawing your holes and other relevant geometry, marking out the outer boundary using a second set of circles, and then connecting the boundary with tangent lines.

In [293]:
%%manim -v WARNING --disable_caching -$quality IntakePlateScene

class IntakePlateScene(Scene):
    def construct(self):
        smallBase = PlateCircle(0.15, 0.2)
        mediumBase = PlateCircle(0.4, 0.2)

        front_hole = np.array([-4, -3, 0])
        middle_hole = np.array([-1.5, 0.25, 0])
        back_hole = np.array([2.5, 1.5, 0])
        back_offset = np.array([0.8, 0.75, 0])

        points = [
            mediumBase.copy(front_hole),
            mediumBase.copy(middle_hole),
            mediumBase.copy(back_hole),
            smallBase.copy(back_hole + back_offset),
            smallBase.copy(back_hole + np.array([1, -0.2, 0])),
            smallBase.copy((middle_hole + back_hole) / 2),
            smallBase.copy((front_hole + middle_hole) / 2),
        ]

        boundary_order = [0, 4, 3, 1]
        plate = Plate(points, boundary_order)

        title = Text("1. Draw plate holes", font_size=MED_FONT)
        title.to_corner(UP + LEFT)
        title_2 = Text("2. Add larger circles", font_size=MED_FONT)
        title_2.to_corner(UP + LEFT)
        title_3 = Text("3. Connect boundary", font_size=MED_FONT)
        title_3.to_corner(UP + LEFT)
        title_4 = Text("4. Cleanup", font_size=MED_FONT)
        title_4.to_corner(UP + LEFT)

        self.play(Write(title), run_time=0.5)

        self.play(plate.draw_inner_circles())
        self.wait(1)

        self.play(Transform(title, title_2), run_time=1)
        self.play(plate.draw_outer_circles())
        self.wait(1)

        self.play(Transform(title, title_3), run_time=1)
        self.play(plate.draw_boundary())
        self.wait(3)

In the event a hole is too close to the edge of a plate, you may need to redraw the boundary.

In [239]:
%%manim -v WARNING $quality BoundaryRedrawScene

class BoundaryRedrawScene(Scene):
    def construct(self):
        left = PlateCircle(1.75, 0.75)
        right = PlateCircle(1.75, 0.75)
        left.move_to([-6, -2, 0])
        right.move_to([6, -2, 0])
        self.add(left)
        self.add(right)

        line = PlateCircle.tangent_line(right, left)
        self.add(line)

        middle = PlateCircle(1, 0.75)
        middle.move_to([0, -.75, 0])
        self.add(middle.inner_circle())

        title = Text("1. Add outer circle", font_size=MED_FONT)
        title.to_corner(LEFT + UP)
        self.play(Write(title))
        self.play(middle.draw_outer_circle())

        title_2 = Text("2. Redraw boundary", font_size=MED_FONT)
        title_2.to_corner(LEFT + UP)
        self.play(Transform(title, title_2))

        self.play(Uncreate(line))
        self.play(Create(PlateCircle.tangent_line(right, middle)))
        self.play(Create(PlateCircle.tangent_line(middle, left)))
        self.wait(3)

In [282]:
class LineSegment(VGroup):
    def __init__(self, start_point: np.array, end_point: np.array) -> None:
        self._line = Line(start_point, end_point)
        self._start = Dot(start_point)
        self._end = Dot(end_point)
        super().__init__(self._line, self._start, self._end)

    def line(self) -> Line:
        return self._line

    def start_point(self) -> Dot:
        return self._start

    def end_point(self) -> Dot:
        return self._end

    def create(self) -> Animation:
        return Succession(Create(self.start_point(), run_time=0.05), Create(self._line), Create(self.end_point(), run_time=0.05))
    
    def move_start(self, new_start_point: np.array, **kwargs) -> Animation:
        return Transform(self, LineSegment(new_start_point, self._end.get_center()), **kwargs)

    def move_end(self, new_end_point: np.array, **kwargs) -> Animation:
        return Transform(self, LineSegment(self._start.get_center(), new_end_point), **kwargs)

In [285]:
%%manim -v WARNING -$quality LineConstraintScene

class LineConstraintScene(Scene):
    def construct(self):
        left = PlateCircle(1.75, 0.75)
        right = PlateCircle(1.75, 0.75)
        left.move_to([-6, -2, 0])
        right.move_to([6, -2, 0])
        self.add(left)
        self.add(right)

        tangent_points = PlateCircle.tangent_points(right, left)

        title = Text("1. Create line", font_size=MED_FONT)
        title.to_corner(LEFT + UP)
        self.play(Write(title))

        left_start_point = tangent_points[1] + [1.75, 0.75, 0]
        right_start_point = tangent_points[0] + [-2, 0.5, 0]
        line = LineSegment(left_start_point, right_start_point)

        self.play(line.create())

        title_2 = Text("2. Add coincident constraints", font_size=MED_FONT)
        title_2.to_corner(LEFT + UP)
        self.play(Transform(title, title_2))

        left_coincident_point = left.center() + normalize(left_start_point - left.center()) * left.outer_radius()
        self.flash(line.start_point(), left)
        self.play(line.move_start(left_coincident_point))

        right_coincident_point = right.center() + normalize(right_start_point - right.center()) * right.outer_radius()
        self.flash(line.end_point(), right)
        self.play(line.move_end(right_coincident_point))


        title_3 = Text("3. Add tangent constraints", font_size=MED_FONT)
        title_3.to_corner(LEFT + UP)
        self.play(Transform(title, title_3))

        self.flash(line.start_point(), left)
        angle = angle_between_vectors(left_coincident_point - left.center(), tangent_points[1] - left.center())
        self.play(line.move_start(tangent_points[1]), path_arc=angle, path_arg_centers=[left.center()])

        self.flash(line.end_point(), right)
        angle = -angle_between_vectors(right_coincident_point - right.center(), tangent_points[0] - right.center())
        self.play(line.move_end(tangent_points[0]), path_arc=angle, path_arg_centers=[right.center()])

        self.wait(3)
    
    def flash(self, point, circle):
        self.play(Flash(point), run_time=0.75)
        self.play(Flash(circle, flash_radius=circle.outer_radius(), num_lines=40), run_time=0.75)
